In [ ]:
%cd ..

In [ ]:
# Deleting the .apperception_cache if it exists, as to avoid DB conflict errors
import os
import shutil

dirpath = os.path.join('.apperception_cache')
if os.path.exists(dirpath) and os.path.isdir(dirpath):
    shutil.rmtree(dirpath)

dirpath = os.path.join('output')
if os.path.exists(dirpath) and os.path.isdir(dirpath):
    shutil.rmtree(dirpath)
os.mkdir(dirpath)

# This piece of code is unsafe, and should not be run if not needed. 
# It serves for test purposes when one recieves a "dead kernel" error.
# os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [ ]:
import os
import sys
import json
import pandas as pd
import cv2
import psycopg2
import pickle
sys.path.append(os.path.join(os.getcwd(),"apperception"))

In [ ]:
from new_world import empty_world, World
from apperception.data_types import Camera
from apperception.utils import scenic_generate_df

In [ ]:
World.db.reset()

In [ ]:
# conn = psycopg2.connect(database="mobilitydb", user="docker", password="docker", host="localhost", port=25432)
# import_tables(conn)

In [ ]:
name = 'ScenicWorld' # world name
units = 'metrics'      # world units
user_data_dir = os.path.join("v1.0-mini")

In [ ]:
# Either uncomment this cell
# df_sample_data, df_annotation = scenic_generate_df()
# with open('df_sample_data.pickle', "wb") as f:
#     pickle.dump(df_sample_data, f)
# with open('df_annotation.pickle', "wb") as f:
#     pickle.dump(df_annotation, f)

In [ ]:
# Or uncomment this cell
with open('df_sample_data.pickle', "rb") as f:
    df_sample_data = pickle.loads(f.read())
with open('df_annotation.pickle', "rb") as f:
    df_annotation = pickle.loads(f.read())

In [ ]:
# df_sample_data

In [ ]:
# df_annotation

In [ ]:
world = empty_world(name=name)

In [ ]:
# cam_x, cam_y = 1317, 1463
# world.road_direction(cam_x, cam_y)

In [ ]:
from apperception.utils import df_to_camera_config
# scenes = ["scene-0061", "scene-0103","scene-0553", "scene-0655", "scene-0757", "scene-0796", "scene-0916", "scene-1077", "scene-1094", "scene-1100"]
scenes = ["scene-0061"]
for scene in scenes:
    config = df_to_camera_config(scene, df_sample_data)
    camera = Camera(config=config, id=scene)
    world = world.add_camera(camera)
    # df_config = df_sample_data[df_sample_data['scene_name'] == scene][['sample_token']]
    # df_ann = df_annotation.join(df_config.set_index('sample_token'), on='sample_token', how='inner')
    world = world.recognize(camera, df_annotation)

In [ ]:
car_trajectories = world.get_traj()

In [ ]:
sum(sum(tt[0] for tt in t[0][0]['coordinates']) / len(t[0][0]['coordinates']) for t in car_trajectories) / len(car_trajectories)

In [ ]:
world.exclude(world.filter(lambda o, c, r: ST_X(valueAtTimestamp(o.trajCentroids, c.timestamp)) < 396.2309651419831)).get_traj()